In [1]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
from keras.layers import Dropout

In [2]:
data = pd.read_csv('Données élections et INSEE.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
data = data.loc[data['Tentative de réélection']=='True']

/var/folders/99/gq6w32xj1_b7n4b18rr2czgr0000gn/T/ipykernel_2765/2271659160.py:1: DtypeWarning: Columns (1,1557,1558,1559) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Données élections et INSEE.csv')


In [3]:
data['y'] = data['Nom de l\'élu en 2014'] == data['Nom de l\'élu en 2020']
data['y'] = data['y'].apply(lambda row: int(row))
y = data['y']

cols = ['CODGEO', 'Nom de l\'élu en 2020', 'Nom de l\'élu en 2014', 'Libellé de la commune',
       'Tentative de réélection', 'y']
cols2 = list(data.columns)
for i in cols:
    cols2.remove(i)
    
X = data[cols2]

In [4]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameters to optimize
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30],
    'units_1': [16, 32, 64],
    'units_2': [8, 16, 32],
    'dropout': [0.2, 0.4, 0.6]
}

# Define the model
def create_model(optimizer, batch_size, epochs, units_1, units_2, dropout):
    model = Sequential()
    model.add(Dense(units=units_1, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Use RandomizedSearchCV to find the best combination of parameters
search = RandomizedSearchCV(estimator=create_model, param_distributions=param_grid, n_iter=10, cv=3, scoring='accuracy', verbose=1)

# Train the model using the best combination of parameters
if len(X) > 10000:
    search.fit(X_train, y_train, batch_size=128, epochs=30)
else:
    search.fit(X_train, y_train)

# Make predictions on the test data
y_pred = search.predict(X_test)


TypeError: estimator should be an estimator implementing 'fit' method, <function create_model at 0x1032e3eb0> was passed

In [69]:
from keras.models import clone_model

# Define the parameters to optimize
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30],
    'units_1': [16, 32, 64],
    'units_2': [8, 16, 32],
    'dropout': [0.2, 0.4, 0.6]
}

# Define the model
def create_model(optimizer, batch_size, epochs, units_1, units_2, dropout):
    model = Sequential()
    model.add(Dense(units=units_1, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create an instance of the Keras model
model = create_model(optimizer='adam', batch_size=32, epochs=10, units_1=16, units_2=8, dropout=0.2)

# Use the clone_model() function to create a clone of the model
model_clone = clone_model(model)

# Use RandomizedSearchCV to find the best combination of parameters
search = RandomizedSearchCV(estimator=model_clone, param_distributions=param_grid, n_iter=10, cv=3, scoring='accuracy', verbose=1)

# Train the model using the best combination of parameters
if len(X) > 10000:
    search.fit(X_train, y_train, batch_size=128, epochs=30)
else:
    search.fit(X_train, y_train)

# Make predictions on the test data
y_pred = search.predict(X_test)


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x296bf55a0>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.